# *E. coli* Live Simulation Notebook

## Setup Simulation

In [1]:
import os
import sys
import datetime

WC_ECOLI_DIRECTORY = os.path.expanduser("~/wcEcoli")
OUT_DIRECTORY = os.path.join(WC_ECOLI_DIRECTORY, "out")

now = datetime.datetime.now()
SUBMISSION_TIME = "%04d%02d%02d.%02d%02d%02d.%06d" % (
	now.year, now.month, now.day,
	now.hour, now.minute, now.second,
	now.microsecond)

JUPYTER_OUT_DIR = "jupyter"
KB_DIRECTORY = os.path.join(OUT_DIRECTORY, JUPYTER_OUT_DIR, "kb")
METADATA_DIRECTORY = os.path.join(OUT_DIRECTORY, JUPYTER_OUT_DIR, "metadata")
CELL_DIRECTORY = os.path.join(OUT_DIRECTORY, JUPYTER_OUT_DIR, "wildtype_000000", "000000", "generation_000000", "000000")
SIM_DATA_PATH = os.path.join(KB_DIRECTORY, "simData_Fit_1.cPickle")

if not os.path.exists(KB_DIRECTORY):
    os.makedirs(KB_DIRECTORY)

if not os.path.exists(METADATA_DIRECTORY):
    os.makedirs(METADATA_DIRECTORY)
    
if not os.path.exists(CELL_DIRECTORY):
    os.makedirs(CELL_DIRECTORY)

sys.path.append(WC_ECOLI_DIRECTORY)

Load and fit the data.

In [2]:
import cPickle
from reconstruction.ecoli.knowledge_base_raw import KnowledgeBaseEcoli
from reconstruction.ecoli.fit_sim_data_1 import fitSimData_1
from reconstruction.ecoli.fit_sim_data_2 import fitSimData_2

raw_data = KnowledgeBaseEcoli()
sim_data = fitSimData_1(raw_data)

with open(SIM_DATA_PATH, 'wb') as f:
    cPickle.dump(sim_data, f, cPickle.HIGHEST_PROTOCOL)

/share/PI/mcovert/pyenv/versions/wcEcoli/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [8]:
from models.ecoli.sim.simulation import EcoliSimulation

class EcoliLiveSimulation(EcoliSimulation):
    def startup(self):
        # Perform initial mass calculations
        for state in self.states.itervalues():
            state.calculatePreEvolveStateMass()
            state.calculatePostEvolveStateMass()

        # Perform initial listener update
        for listener in self.listeners.itervalues():
            listener.initialUpdate()

        # Start logging
        for logger in self.loggers.itervalues():
            logger.initialize(self)
            
    def step(self, steps=1, time=None):
        startTime = self._timeTotal
        stepCount = 0
        
        while True:
            stepCount += 1
            
            if steps > 0 and stepCount > steps:
                return
            
            if time is not None and startTime + time < self._timeTotal:
                return
            
            if self._cellCycleComplete:
                print "Cell cycle complete"
                return

            self._simulationStep += 1
            self._timeTotal += self._timeStepSec
            self._evolveState()
    
    def shutdown(self):
        # Run post-simulation hooks
        for hook in self.hooks.itervalues():
            hook.finalize(self)

        # Divide mother into daughter cells
        self._divideCellFunction()

        # Finish logging
        for logger in self.loggers.itervalues():
            logger.finalize(self)

In [4]:
options = {}

options["simDataLocation"] = SIM_DATA_PATH
options["outputDir"] = os.path.join(CELL_DIRECTORY, "simOut")
sim = EcoliLiveSimulation(**options)

In [6]:
sim.startup()

Time (s)  Dry mass     Dry mass      Protein          RNA     Expected
              (fg)  fold change  fold change  fold change  fold change
========  ========  ===========  ===========  ===========  ===========
       0    245.46        1.000        1.000        1.000        1.000


In [10]:
sim.step(steps=1)

      11    246.93        1.006        1.002        1.000        1.002
steps
